### Run this file before running adaboost.ipynb:

In [2]:
import cv2
import imutils
import matplotlib
import numpy as np
import pandas as pd

In [3]:
with open('../faces.txt') as fp:
    faces_files_path = fp.readlines()
with open('../non-faces.txt') as nfp:
    nonfaces_files_path = nfp.readlines()
    
Feature1_indices = pd.read_csv('Int_image_indices/Feature1_indices.csv')
Feature2_indices = pd.read_csv('Int_image_indices/Feature2_indices.csv')
Feature3_indices = pd.read_csv('Int_image_indices/Feature3_indices.csv')
Feature4_indices = pd.read_csv('Int_image_indices/Feature4_indices.csv')
Feature5_indices = pd.read_csv('Int_image_indices/Feature5_indices.csv')

In [8]:
#Integral Image access function
def getIntImageValues(Int_image, index_i,index_j):
            index_valid = (index_i>=0)*(index_j>=0)
            index_i_valid=(index_valid*index_i).astype(int)
            index_j_valid=(index_valid*index_j).astype(int)
            return Int_image[index_i_valid,index_j_valid]-Int_image[0,0]*np.invert(index_valid)
A=getIntImageValues
detectorWidth=19
w = detectorWidth

In [4]:
def getFeature1Values(Int_image, K_min,K_max,J_min,J_max,J_min_up,J_max_up):
            Bottom_area=A(Int_image,J_max,K_max)-A(Int_image,J_max,K_min-1)-\
A(Int_image,J_min-1,K_max)+A(Int_image,J_min-1,K_min-1)
            Top_area=A(Int_image,J_max_up,K_max)-A(Int_image,J_max_up,K_min-1)-\
A(Int_image,J_min_up-1,K_max)+A(Int_image,J_min_up-1,K_min-1)
            return Bottom_area-Top_area
def getFeature2Values(Int_image, K_min,K_max,J_min,J_max,K_min_right,K_max_right):
            Left_area=A(Int_image,J_max,K_max)-A(Int_image,J_max,K_min-1)-\
A(Int_image,J_min-1,K_max)+A(Int_image,J_min-1,K_min-1)
            Right_area=A(Int_image,J_max,K_max_right)-A(Int_image,J_max,K_min_right-1)-\
A(Int_image,J_min-1,K_max_right)+A(Int_image,J_min-1,K_min_right-1)
            return Right_area-Left_area
def getFeature3Values(Int_image, K_min,K_max,K_min_middle,K_max_middle,K_min_right,K_max_right,J_min,J_max):
            Left_area=A(Int_image,J_max,K_max)-A(Int_image,J_max,K_min-1)-\
A(Int_image,J_min-1,K_max)+A(Int_image,J_min-1,K_min-1)
            Middle_area=A(Int_image,J_max,K_max_middle)-A(Int_image,J_max,K_min_middle-1)-\
A(Int_image,J_min-1,K_max_middle)+A(Int_image,J_min-1,K_min_middle-1)
            Right_area=A(Int_image,J_max,K_max_right)-A(Int_image,J_max,K_min_right-1)-\
A(Int_image,J_min-1,K_max_right)+A(Int_image,J_min-1,K_min_right-1)
            return Right_area+Left_area-Middle_area
def getFeature4Values(Int_image, J_min,J_max,J_min_middle,J_max_middle,J_min_up,J_max_up,K_min,K_max):
            Top_area=A(Int_image,J_max_up,K_max)-A(Int_image,J_max_up,K_min-1)-\
A(Int_image,J_min_up-1,K_max)+A(Int_image,J_min_up-1,K_min-1)
            Middle_area=A(Int_image,J_max_middle,K_max)-A(Int_image,J_max_middle,K_min-1)-\
A(Int_image,J_min_middle-1,K_max)+A(Int_image,J_min_middle-1,K_min-1)
            Bottom_area=A(Int_image,J_max,K_max)-A(Int_image,J_max,K_min-1)-\
A(Int_image,J_min-1,K_max)+A(Int_image,J_min-1,K_min-1)
            return Top_area+Bottom_area-Middle_area
def getFeature5Values(Int_image, K_min,K_max,J_min,J_max,K_min_right,K_max_right,J_min_up,J_max_up):
            TopLeft_area=A(Int_image,J_max_up,K_max)-A(Int_image,J_max_up,K_min-1)-\
A(Int_image,J_min_up-1,K_max)+A(Int_image,J_min_up-1,K_min-1)
            TopRight_area=A(Int_image,J_max_up,K_max_right)-A(Int_image,J_max_up,K_min_right-1)-\
A(Int_image,J_min_up-1,K_max_right)+A(Int_image,J_min_up-1,K_min_right-1)
            BottomLeft_area=A(Int_image,J_max,K_max)-A(Int_image,J_max,K_min-1)-\
A(Int_image,J_min-1,K_max)+A(Int_image,J_min-1,K_min-1)
            BottomRight_area=A(Int_image,J_max,K_max_right)-A(Int_image,J_max,K_min_right-1)-\
A(Int_image,J_min-1,K_max_right)+A(Int_image,J_min-1,K_min_right-1)
            return TopLeft_area+BottomRight_area-TopRight_area-BottomLeft_area


In [5]:
def find_features(image_path,detectorWidth=19):
    img=cv2.imread(image_path,cv2.IMREAD_GRAYSCALE)
    img = np.asarray(imutils.resize(img, width=detectorWidth),dtype="int64")
    Matrix_S=np.reshape([np.sum(img[range(j+1),:],axis=0) 
                         for j in range(detectorWidth)],(detectorWidth,detectorWidth))
    Int_image=np.transpose(np.reshape([np.sum(Matrix_S[:,range(j+1)],axis=1) 
                                   for j in range(detectorWidth)],(detectorWidth,detectorWidth)))
    Feature1=getFeature1Values(Int_image, np.array(Feature1_indices.K_min),np.array(Feature1_indices.K_max)\
                           ,np.array(Feature1_indices.J_min),np.array(Feature1_indices.J_max),\
                           np.array(Feature1_indices.J_min_up),np.array(Feature1_indices.J_max_up) )
    Feature2=getFeature2Values(Int_image, np.array(Feature2_indices.K_min),np.array(Feature2_indices.K_max)\
                           ,np.array(Feature2_indices.J_min),np.array(Feature2_indices.J_max),\
                           np.array(Feature2_indices.K_min_right),np.array(Feature2_indices.K_max_right) )
    Feature3=getFeature3Values(Int_image, np.array(Feature3_indices.K_min),np.array(Feature3_indices.K_max)\
                           ,np.array(Feature3_indices.K_min_middle),np.array(Feature3_indices.K_max_middle)\
                           ,np.array(Feature3_indices.K_min_right),np.array(Feature3_indices.K_max_right)\
                           ,np.array(Feature3_indices.J_min),np.array(Feature3_indices.J_max) )
    Feature4=getFeature4Values(Int_image, np.array(Feature4_indices.J_min),np.array(Feature4_indices.J_max)\
                           ,np.array(Feature4_indices.J_min_middle),np.array(Feature4_indices.J_max_middle)\
                           ,np.array(Feature4_indices.J_min_up),np.array(Feature4_indices.J_max_up)\
                           ,np.array(Feature4_indices.K_min),np.array(Feature4_indices.K_max) )
    Feature5=getFeature5Values(Int_image, np.array(Feature5_indices.K_min),np.array(Feature5_indices.K_max)\
                           ,np.array(Feature5_indices.J_min),np.array(Feature5_indices.J_max)\
                           ,np.array(Feature5_indices.K_min_right),np.array(Feature5_indices.K_max_right)\
                           ,np.array(Feature5_indices.J_min_up),np.array(Feature5_indices.J_max_up) )
    return np.concatenate((Feature1,Feature2,Feature3,Feature4,Feature5))

### Change data location in below cell:

In [9]:
features_faces=np.vstack([find_features(str('../dataset/trainset/faces/'+image_path).rstrip("\n\r")) 
           for image_path in faces_files_path])

In [10]:
np.save('features_faces.npy', features_faces)

### Change data location in below cell:

In [ ]:
features_nonfaces=np.vstack([find_features(str('../dataset/trainset/non-faces/'+image_path).rstrip("\n\r"))
                          for image_path in nonfaces_files_path])

In [ ]:
np.save('features_nonfaces.npy', features_nonfaces)